# American Sign Language (ASL) Image Recognition

## Introduction

## Loading in dataset

In [ ]:
import numpy as np
from sklearn.datasets import load_files

# 
def load_dataset(path):
    data = load_files(path)
    image_files = np.array(data['filenames'])
    return dog_files

data = load_dataset('data')

### Dividing data into training, validation, and testing sets

Now that preprocessing the images is completed (see `data_preprocessing.ipynb` notebook), the full dataset will be split into training, validation, and testing sets. The testing set will be all the images from one subject to mirror the "Spelling It Out" paper's method so the benchmark model can be compared. The rest of the images will be randomly split; 80% of images for training, 20% of the images for validation.

## Building the model

## Training the model

## Evaluating the model